# CoCO - your sophisticated and lovely tiny Commerzbank Chat Operator

<img src="img/coco_avatar.jpg" height="100" />

## Demo Commerzbank - rasa - Account ChatBot
Hi I'm CoCO your Commerzbank Chat Operator. I'm able to assist you in all questions around your account. But I'm new in Commerzbank and I do not know everything. You should be patient and help me to learn this.

At the beginning I only now topics around
* asking for your name. 
* asking for your accounts
* get a list of all accounts
* your account balance
* your maximum debit
* I can give you your IBAN per account
* Bot-Context:
    - username
    - account number

If you help me, I will learn more - trust me -


# CoCO is based on rasa_nlu - Natural language understanding

This notebook is based on a local configured machine with below pre installations available
* Python 3.6.x (do not install Python 3.7, because (JUL18), there is no TensorFlow package available
* Latest version from rasa_core (rasa.com)
* Latest version from rasa_nlu (rasa.com)
* Latest version from spaCy
* Tensorflow (should be installed with rasa_core)
* Keras (should be installed with rasa_core)

In [ ]:
# use this requirement file if you have to install all packages again
#!pip3 install -r requirements.txt

### recommandation - download medium size language model (_md) instead of _sm)

you should download language files only once - for this demo ONLY english is used

In [ ]:
# Language files are > 100MB

# ENGLISH
#python3 -m spacy download en_core_web_md
#python3 -m spacy link en_core_web_md en

# GERMAN
#python3 -m spacy download de_core_web_md
#python3 -m spacy link de_core_web_md de

# MULTI
#python3 -m spacy download xx_core_web_md
#python3 -m spacy link xx_core_web_md xx


next part is only for checking, if environment is set to Python 3.6

In [ ]:
import sys
sys.path

In [ ]:
!python3 --version

In [ ]:
!mkdir data

Only a test if rasa core & nlu is available. Result should 2x 0

In [ ]:
!python3 -c "import rasa_core"
!echo $?

!python3 -c "import rasa_nlu"
!echo $?

In [ ]:
import logging, io, json, warnings
logging.basicConfig(level="INFO")
warnings.filterwarnings('ignore')

# NLU-TRAINING Step 1 (DATA) 
## Let's  start with a data.json file - contain intents, entities, ...

If we build a NLU-ChatBot, one of the first things is to create a data file which contain intents and entities (and later on much more).

In this first step, the file contain only some greetings and some intents around the account number

To create below file, you can use **rasa-nlu-trainer*** but before you can do this, you need node.js and install via npm this trainer

If you got this: "mkdir: data: File exists" - don't worry, the folder is already existing :-)

#### Please go to the overview page and check in data/coco_data.json file

This file contain current intents and entities. It's to big to show it here

### Short introductin into the structure

    {
        {
            "text": "Hello",
            "intent": "greet",
            "entities": []
          },
    }

This is a simple intent with no entities. Predicted user text is **Hello** and the intent behind is **greet**.
That means, if user enter **Hello**, nlu map this input the the intent **greet**

      {
        "text": "what is my account number?",
        "intent": "user_ask_accountnumbers",
        "entities": [
          {
            "start": 11,
            "end": 25,
            "value": "account number",
            "entity": "accountnumber"
          }
        ]
      },

This is a more complicated intent. User input **what is my ....**, is mapped to the intent **user_ask_accountnumbers**. So know, we know user ask something around account numbers. The entity is in this case the part **account number** in users input text and is mapped to the entity **accountnumber**.


# TRAINING Step 2 (CONFIG)
## creating a configuration file for CoCO

this configuration file contain some important information for the AI behind the chatbot and are used during trainging.

We name this file config_spacy.json

### Processing Pipeline
The process of incoming messages is split into different components. These components are executed one after another in a so called processing pipeline. There are components for entity extraction, for intent classification, pre-processing and there will be many more in the future.

Each component processes the input and creates an output. The ouput can be used by any component that comes after this component in the pipeline. There are components which only produce information that is used by other components in the pipeline and there are other components that produce Output attributes which will be returned after the processing has finished. For example, for the sentence "I am looking for Chinese food" the output

In the pipeline you describe which featurizer, feature extractor, tokenizer you want to use to analyse you text. The result from one component is send to the next component in the pipeline.

<img src="img/component_lifecycle.png"/>


In [ ]:
config = """
language : "en"

pipeline:
- name: "nlp_spacy"
- name: "tokenizer_spacy"
- name: "intent_featurizer_spacy"
- name: "intent_classifier_sklearn"
"""

%store config > config_spacy.yml

# TRAIING Step 3a (training)
## creating a nlu_model.py file

This file is needed to load the model and train the bot

In [1]:
#%%writefile nlu_model.py

# if you change something in this cell please remove from next line the #, run cell and afterwards insert # again
#

# this is needed to load our training dataset
from rasa_nlu.training_data import load_data
# this is needed to load our configuration file
from rasa_nlu.config import RasaNLUModelConfig
from rasa_nlu import config
# this is our Model-Trainer
from rasa_nlu.model import Trainer

def training_nlu (data, config_spacy, model_path):
    training_data = load_data(data)
    # this is our Trainer, he's responsible to train the bot due to training data
    #trainer = Trainer(RasaNLUModelConfig({"pipeline": pipeline}))
    trainer = Trainer(config.load(config_spacy))

    # start training
    interpreter = trainer.train(training_data)
    # persist the training result
    model_directory = trainer.persist(model_path, fixed_model_name = "coconlu")
   

if __name__ == '__main__':
    training_nlu('./data/coco_data.json', "./config_spacy.yml", './models/nlu')
    


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Fitting 2 folds for each of 6 candidates, totalling 12 fits


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in l

### Check if model files are available

you have to go to Jupyter overview (click above on jupyter logo)
Go to subfolder **models/nlu/default**
System should created a new folder **coconlu** with following files
* intent_classifier_sklearn.pkl
* metadata.json
* training_data.json

# TRAINING Step 3b ( build a model)

In [3]:
#%%writefile nlu_model.py

# if you change something in this cell please remove from next line the #, run cell and afterwards insert # again
#

# this is needed to load our training dataset
from rasa_nlu.training_data import load_data
# this is needed to load our configuration file
from rasa_nlu.config import RasaNLUModelConfig
from rasa_nlu import config
# this is our Model-Trainer
from rasa_nlu.model import Trainer
from rasa_nlu.model import Metadata, Interpreter

def training_nlu (data, config_spacy, model_path):
    training_data = load_data(data)
    # this is our Trainer, he's responsible to train the bot due to training data
    #trainer = Trainer(RasaNLUModelConfig({"pipeline": pipeline}))
    trainer = Trainer(config.load(config_spacy))

    # start training
    trainer.train(training_data)
    # persist the training result
    model_directory = trainer.persist(model_path, fixed_model_name = "coconlu")
 
def run_nlu():
    interpreter = Interpreter.load('./models/nlu/default/coconlu')
    print (interpreter.parse(u"I would like to have my account number"))
    
    
if __name__ == '__main__':
    training_nlu('./data/coco_data.json', "./config_spacy.yml", './models/nlu')
    run_nlu()



Fitting 2 folds for each of 6 candidates, totalling 12 fits


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in l

{'intent': {'name': 'user_ask_accountnumbers', 'confidence': 0.4601363274961629}, 'entities': [], 'intent_ranking': [{'name': 'user_ask_accountnumbers', 'confidence': 0.4601363274961629}, {'name': 'chatbot_need_username', 'confidence': 0.35950975468983354}, {'name': 'goodbye', 'confidence': 0.1166203262914735}, {'name': 'greet', 'confidence': 0.06373359152253019}], 'text': 'I would like to have my account number'}


### Happy days - our first response was send back from the bot

* {'intent': {'name': **'accountnumber'**, 'confidence': 0.6448104231702032}, 'entities': [] ...
    - Yes, NLU recognized, that user want someting about account number

* {'intent': {'name': 'accountnumber', **'confidence': 0.6448104231702032}**, 'entities': [],...
    - We got a prediction of approx 65%

* {'intent': {'name': 'accountnumber', 'confidence': 0.6448104231702032}, **'entities': []** ...
    - No entities found

The Rest of the json file are predictions of other intents with less percentage

### Let's test with a more or less exakt question 

In [ ]:
#%%writefile nlu_model.py

# if you change something in this cell please remove from next line the #, run cell and afterwards insert # again
#

# this is needed to load our training dataset
from rasa_nlu.training_data import load_data
# this is needed to load our configuration file
from rasa_nlu.config import RasaNLUModelConfig
from rasa_nlu import config
# this is our Model-Trainer
from rasa_nlu.model import Trainer
from rasa_nlu.model import Metadata, Interpreter

def training_nlu (data, config_spacy, model_path):
    training_data = load_data(data)
    # this is our Trainer, he's responsible to train the bot due to training data
    #trainer = Trainer(RasaNLUModelConfig({"pipeline": pipeline}))
    trainer = Trainer(config.load(config_spacy))

    # start training
    trainer.train(training_data)
    # persist the training result
    model_directory = trainer.persist(model_path, fixed_model_name = "coconlu")
 
def run_nlu():
    interpreter = Interpreter.load('./models/nlu/default/coconlu')
    print (interpreter.parse(u"get my account number"))
    
    
if __name__ == '__main__':
    training_nlu('./data/coco_data.json', "./config_spacy.yml", './models/nlu')
    run_nlu()



In [ ]:
#%%writefile nlu_model.py

# overwrite current nlu_model.py with this version 

# this is needed to load our training dataset
from rasa_nlu.training_data import load_data
# this is needed to load our configuration file
from rasa_nlu.config import RasaNLUModelConfig
from rasa_nlu import config
# this is our Model-Trainer
from rasa_nlu.model import Trainer
from rasa_nlu.model import Metadata, Interpreter

def training_nlu (data, config_spacy, model_path):
    training_data = load_data(data)
    # this is our Trainer, he's responsible to train the bot due to training data
    #trainer = Trainer(RasaNLUModelConfig({"pipeline": pipeline}))
    trainer = Trainer(config.load(config_spacy))

    # start training
    trainer.train(training_data)
    # persist the training result
    model_directory = trainer.persist(model_path, fixed_model_name = "coconlu")
 
def run_nlu():
    interpreter = Interpreter.load('./models/nlu/default/coconlu')
    print (interpreter.parse(u"hello"))
    
    
if __name__ == '__main__':
    training_nlu('./data/coco_data.json', "./config_spacy.yml", './models/nlu')
    run_nlu()



# Creating a dialog for CoCO - rasa_core

## STEP 1  (DOMAIN) - we create a dialog flow for CoCO

first we have to create a domain specific file. A domain is coco's universe or world. In this domain, CoCO can act and use actions during a conversation with the user.

### Step 1a - creating a domain specific file

A domain contain five different parts
* slots
    - Slots are the storage space for data from an entity. They store the context of the chat.
        - Example: accountnumber: - type: text
    The slot is "accountnumber" and the value is a text-value (alphanumeric)
    - Via a tracker, we can get this information inside our application logic. tracker.get_slot("accountnumber")

* intents
    - Intents are parts of a conversation, which the bot should expected to response to. Inside NLU, this is the first step. The sentence will be categorized in an **intend**
        - Example: "Hello CoCO" (user input) refers to intent: "intent_greet",
        "I need my account number" => "intent_accountnumber"
    - The intent describe what the user probably meant to say
* entities
    - pieces of info you want to extract from messages. This is inside NLU the second step. Words in the sentence are categorized into **entities**. Like **accountnumber**
* templates
    - Utterance templates are messages the bot will send back to the user.
    Either automatically by an action with the same name as the utterance or by an action with custom code.

* actions
    - Actions are things, which can CoCO do. Like calling an API, reading from a database, searching some in the background. Allways an action reflects to an specific function. 
    - Actions and templates **should use the same names**
    - Actions can do:
        - response to a user
        - call an API / Service / ...
        - query database



In [ ]:
domain = """
intents:
- greet
- goodbye
- user_need_help
- chatbot_need_username


entities:
- accountnumber
- username

actions:
- utter_greet
- utter_goodbye
- utter_chatbot_need_username
- utter_user_need_help
- utter_user_ask_accountnumbers
- actions.ActionAccountNumber
- actions.ActionHelp
- actions.ActionUsername

templates:
    utter_greet:
        - 'Hi, I am CoCO your virtual operator. What can I do for your?'
        - 'Nice to meet you. Here is CoCO your virtal operator. What are your interested in?'
        - 'This is CoCO your sophisticated virtual operator. What would you like to do?'
        - 'Hey What would you like to do?'

    utter_goodbye:
        - 'It was a pleasure to meet you. Good bye'
        - 'See you later'
        - 'Hope I could help you. Bye'
        - 'Bye bye'
        - ' Cu u later aligator'
        
    utter_chatbot_need_username:
        - 'For saftey issues, I need your first name'
        - 'Mhm, maybe I do not know you. Please enter your first name'
        - 'Ups, I forgot your name, could you please enter for first name'

    utter_user_need_help:
        - 'You can every type in every time "help". Than I give you a list of possible commands'
        

    utter_user_ask_accountnumbers:
        - 'I will check the system for your account number. Just a moment...'
        - 'I will ask backoffice to get your account number. Be patient...'
        


slots:
    username:
        type: text
    accountnumber:
        type: text

"""

#%store domain > coco_domain.yml

## Step 2 (ACTION) : Defining actions in Python

If you want to implement more complicated actions (like reading the accountnumber and not only answering of something. We need an **actions.py** file and implement this business logic.

If you run this cell, the Account-MOCK will print some information out

In [ ]:
#%%writefile actions.py

# to run this cell insert a # before %%writefile, to store content, remove #


from __future__ import absolute_import
from __future__ import division
from __future__ import unicode_literals

# some imports from rasa
from rasa_core.actions.action import Action
from rasa_core.events import SlotSet

#
# import our Account-MOCK - MOCK do some prints on the screen
import MOCK_AccountDetails

from random import randint, uniform, random

CR = '\n'

class ActionAccountNumber(Action):
    # via this function, we can get the action name.
    # if something is predicted, the chatbot will use this class
    def name(self):
        return 'action_accountnumber'
    
    # if this action should run, this function is called
    def run(self, dispatcher, tracker, domain):
        username = tracker.get_slot('username')
        username = username if username else "ERROR_UNKNOWN_USER"
        #
        # create an account list from 1..10 checking accounts and 1..5 saving accounts
        accounts = MOCK_Response (username, randint(1,10), randint(1,5), CR)
        title = accounts[0]["Title"]
        account_list = CR + accounts[1]
        response = "Hi {} {}, I found follwing accounts : {}".format(title, username, account_list)
        
        #
        # send it out to the user
        dispatcher.utter_message(response)
        return [SlotSet('accounts', accounts)]

class ActionUsername(Action):
    def name(self):
        return 'action_username'
    
    def run(self, dispatcher, tracker, domain):
        username = tracker.get_slot('username')
        username = username if username else "ERROR_UNKNOWN_USER"

        message = "Hi {}, nice to meet you".format(username)
        dispatcher.utter_message(message)
        return [SlotSet('username', username)]

    
class ActionHelp(Action):
    def name(self):
        return 'action_help'
    
    def run(self, dispatcher, tracker, domain):
        username = tracker.get_slot('username')
        username = username if username else "ERROR_UNKNOWN_USER"

        help = 'Following things can I do fo your' + CR
        help += 'ask me for your IBAN'
        help += 'ask me for your banks BIC'
        help += 'I can collect all your accounts'
        help += 'I can give you current balance for an account'
        help += 'I can check your debit for an account'

        dispatcher.utter_message(help)
        return [SlotSet('username', username)]

        
        
        

## Step 3 (STORIES) - Creating a Story for CoCO

Don't worry, this is only a first step. Later on, we create more sophisticated stories

In [ ]:
stories = """

## Story 01
* greet
    - utter_greet
    - utter_ask_username
    - action_username

## Story 02
* goodbye
    - utter_goodbye
        
## Story 04
* user_ask_accountnumbers 
    - utter_ask_accountnumber
    - action_accountnumber

## Story 06
* user_need_help
    - action_help
    
## Story 07
* chatbot_need_username
    - utter_ask_username
    - action_username

"""
%store stories > data/coco_stories.md

# Now we want to train CoCO online (awsome training functionality)



## STEP 1 ONLINE-TRAINING - creating a train_init.py file

Be patient, due to 500 epochs, this can take up to 5mins before system is ready for an online training

## Step 1a - Visualizing your Stories

In [4]:
#%%writefile visualize_coco_dialog.py

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

from rasa_core.agent import Agent
from rasa_core.policies.keras_policy import KerasPolicy
from rasa_core.policies.memoization import MemoizationPolicy

if __name__ == '__main__':
    agent = Agent("coco_domain.yml",
                  policies=[MemoizationPolicy(), KerasPolicy()])

    agent.visualize("data/coco_stories.md",
                    output_file="coco_graph.png", max_history=2)

Using TensorFlow backend.


Checking Account 0
Checking Account 1
Saving Account 0
{'Owner': 'CoCO', 'Title': 'Dr.', 'Accounts': {'Checking_account': [{'Number': '5258508992', 'IBAN': 'DE53300400005258508992', 'Created': '10.01.2010', 'Balance': 11571.48, 'Debit': 1000, 'Currency': '€'}, {'Number': '1889812558', 'IBAN': 'DE53300400001889812558', 'Created': '10.01.2010', 'Balance': 11472.96, 'Debit': 10000, 'Currency': '€'}, {'Number': '9789675264', 'IBAN': 'DE53300400009789675264', 'Created': '10.01.2010', 'Balance': 37320.26, 'Debit': 0, 'Currency': '€'}], 'Saving_account': [{'Number': '5258508992', 'IBAN': 'DE53300400005258508992', 'Created': '10.01.2010', 'Balance': 11571.48, 'Debit': 1000, 'Currency': '€'}, {'Number': '1889812558', 'IBAN': 'DE53300400001889812558', 'Created': '10.01.2010', 'Balance': 11472.96, 'Debit': 10000, 'Currency': '€'}, {'Number': '9789675264', 'IBAN': 'DE53300400009789675264', 'Created': '10.01.2010', 'Balance': 37320.26, 'Debit': 0, 'Currency': '€'}]}, 'DayOfBirth': '04.MAR.1973', 'C

Processed Story Blocks: 100%|██████████| 5/5 [00:00<00:00, 354.01it/s, # trackers=1]


## coco_graph.png
This graph show a dialog flow of your current stories

<img src="./coco_graph.png">

## 1b - Start Trainit initalization

This can take up to several minutes, due to size of domain.yml and stories.md

In [ ]:
#%%writefile train_init.py

from __future__ import absolute_import
from __future__ import division
from __future__ import unicode_literals

import logging

# this is the trainter agent
from rasa_core.agent import Agent
# both imports are the models how to train our model
from rasa_core.policies.keras_policy import KerasPolicy
from rasa_core.policies.memoization import MemoizationPolicy

if __name__ == '__main__':
    logging.basicConfig(level='INFO')
    # our stories for for the trainging
    training_data_file = './data/coco_stories.md'
    # path where files should be stored
    model_path = './models/dialogue'
    # Create a training agent with our domain file and the training models
    agent = Agent('coco_domain.yml', policies = [MemoizationPolicy(max_history = 2), KerasPolicy()])
    # start training, with 500 epochs (iterations), use a batch size of 10, and split trainting data into 80/20
    agent.train(
            training_data_file,
            epochs = 500,
            batch_size = 10,
            validation_split = 0.2)
    # store results       
    agent.persist(model_path)

### Check created model files
If above cell is successfully finished, please switch to jupyter-home and check **models/dialogue**. The train_init.py create a couple of files

# Ready for an online training for CoCO

If you start next cell, the bot is started and wait for user input. You can now test all implemented stories. After every input, system (**CoCO**) return a prediction. You have to check if his prediction is
* ok (YES)
* not ok (NO)
    - decide why it is not ok
    - choose the correct behaviour
    
    
** it can take up to 5min before system is ready for training **

In [ ]:
#%%writefile train_online.py

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import logging

from rasa_core.agent import Agent
from rasa_core.channels.console import ConsoleInputChannel
from rasa_core.interpreter import RegexInterpreter
from rasa_core.policies.keras_policy import KerasPolicy
from rasa_core.policies.memoization import MemoizationPolicy
from rasa_core.interpreter import RasaNLUInterpreter

logger = logging.getLogger(__name__)


def run_coco_online(input_channel, interpreter,
                          domain_file="coco_domain.yml",
                          training_data_file='data/coco_stories.md'):
    agent = Agent(domain_file,
                  policies=[MemoizationPolicy(max_history=2), KerasPolicy()],
                  interpreter=interpreter)

    agent.train_online(training_data_file,
                       input_channel=input_channel,
                       batch_size=50,
                       epochs=200,
                       max_training_samples=300)

    return agent


if __name__ == '__main__':
    #
    # change ERROR to INFO and you see more details for the training phase
    logging.basicConfig(level="INFO")
    nlu_interpreter = RasaNLUInterpreter('./models/nlu/default/coconlu')
    run_coco_online(ConsoleInputChannel(), nlu_interpreter)

## Account-MOCK

This is your test structure, which is used for account numbers. If user take access to the account number, we will generate randomly one or more accounts for the user. Owner will be the user himself. Rest of data is randomly 

    {
      "Owner": "Peter Pan",
      "Title": "PhD",
      "Accounts" : {
        "Checking_account" : [{
          "Number": "3012345678" ,
          "Created" : "01.01.2010",
          "Balance" : 34566.00,
          "Debit" : 10000,
          "Currency" : "€"
          },
          {
          "Number": "3087654321" ,
          "Created" : "01.01.2010",
          "Balance" : 23053.35,
          "Debit" : 15000,
          "Currency" : "€"
          }      
          ],
        "Saving_account"   : [{
          "Number": "9912345678" ,
          "Created" : "01.01.2010",
          "Balance" : 15000.00,
          "Debit" : 0,
          "Currency" : "€"
        }]},
      "DayOfBirth": "04.MAR.1973",
      "CityOfBirth" : "Keras",
      "Address": "Cloudy Street 26a",
      "Zip": 12345,
      "City" : "Heaven",
      "Phone" : ["004032122453"],
      "Mobile" : ["004015175075300"]
    }

In [ ]:
#%%writefile MOCK_AccountDetails.py

from random import randint, uniform, random

CR = '\n'

def MOCK_CreateCoBAAccountID():
    L = [randint(0, 9) for p in range(0, 10)]
    #print (''.join(str(e) for e in L))    
    return "".join(str(e) for e in L)

def MOCK_IBANAccountID():
    account = MOCK_CreateCoBAAccountID()
    return ["DE5330040000" + account, account]

def MOCK_AccountDetails(name, numberOfAccounts, numberOfSavingAccounts):
    titles = ['','Dr.','PhD','Prof', 'God']
    structure = {
                  "Owner": "Peter Pan",
                  "Title": "PhD",
                  "Accounts" : {
                    "Checking_account" : [{
                      "Number": "1234567890" ,
                      "IBAN" : "DE53300400001234567890",
                      "Created" : "10.01.2010",
                      "Balance" : 34566.00,
                      "Debit" : 10000,
                      "Currency" : "€"
                      },
                      {
                      "Number": "0987654321" ,
                      "IBAN" : "DE53300400000987654321",
                      "Created" : "03.05.2011",
                      "Balance" : 23053.35,
                      "Debit" : 15000,
                      "Currency" : "€"
                      }      
                      ],
                    "Saving_account"   : [{
                      "Number": "9912345678" ,
                      "IBAN" : "DE53300400009912345678",
                      "Created" : "10.01.2010",
                      "Balance" : 15000.00,
                      "Debit" : 0,
                      "Currency" : "€"
                    }]},
                  "DayOfBirth": "04.MAR.1973",
                  "CityOfBirth" : "Keras",
                  "Address": "Cloudy Street 26a",
                  "Zip": 12345,
                  "City" : "Heaven",
                  "Phone" : ["004032122453"],
                  "Mobile" : ["004015175075300"]
                }
    structure['Owner'] = name if name else "unknown"
    structure['Title'] = titles[randint(0, 4)]
    #
    # create number of accounts (only checking accounts (Girokonten))
    accounts = []
    
    if numberOfAccounts < 1: 
        numberOfAccounts = 1
    if numberOfAccounts > 10: 
        numberOfAccounts = 10
   
    #
    # create checking accounts
    for x in range (0, numberOfAccounts):
        print ("Checking Account {}".format(x))
        ano = MOCK_IBANAccountID()
        maxDebit = round(randint(1,10),0) * 1000
        s = {
              "Number": ano[1] ,
              "IBAN" : ano[0],
              "Created" : "10.01.2010",
              "Balance" : round(uniform((maxDebit*-1),15000), 2),
              "Debit" : maxDebit,
              "Currency" : "€"            
            } 
        accounts.append(s)
    ##
    ##
    structure['Accounts']['Checking_account'] = accounts
 
    #
    # create saving accounts
    if numberOfSavingAccounts < 1: 
        numberOfSavingAccounts = 1
    if numberOfSavingAccounts > 5: 
        numberOfSavingAccounts = 5
   

    for x in range (0, numberOfSavingAccounts):
        print ("Saving Account {}".format(x))
        ano = MOCK_IBANAccountID()
        s = {
              "Number": ano[1] ,
              "IBAN" : ano[0],
              "Created" : "10.01.2010",
              "Balance" : round(uniform(0,100000), 2),
              "Debit" : 0,
              "Currency" : "€"            
            } 
        accounts.append(s)
    ##
    ##
    structure['Accounts']['Saving_account'] = accounts

    return structure

def MOCK_Response(username, numberOfAccounts, numberOfSavingAccounts, CR):
    accounts = MOCK_AccountDetails(username, numberOfAccounts,numberOfSavingAccounts)
    #print (accounts)

    response = """Checking-Accounts"""
    response +=  CR + "-----------------"
    response += CR
    x=1
    for a in accounts['Accounts']['Checking_account']:
        response += "{:>3}. IBAN: {}, Current Balance: {:>12}{}".format(x, a["IBAN"],  a["Balance"], a['Currency'])
        if a["Balance"] < 0.0:
            response += " Your current debit is max {}{}".format(a['Debit'],a['Currency'])
        response += CR
        x = x+1
    return [accounts, response]


result = MOCK_Response('CoCO',2,1, '\n')
print (result[0])
print (result[1])

# We use TELEGRAM as our chat app

In [ ]:
pip3 install -U python-telegram-bot